# Machine Learning with PyTorch

### How does this lab work ?

The lab uses MNIST datasets. The dataset has over 60.000 images of hand written digits. The data will be partitioned between training the AI model and testing the AI model after training.

The main steps in  this project include:

1. Download the MNIST dataset and create a DataLoader for the dataset.
2. Define an AI model to recognize a hand written the MNIST dataset.
3. Train the defined AI model using training data from the MNIST dataset.
4. Test the trained AI model using testing data from the MNIST dataset.
5. Evaluate

In [ ]:
import torch
print(torch.__version__)
print(torch.nn)

2.9.1+cu128
<module 'torch.nn' from '/home/machine01/.cache/pypoetry/virtualenvs/cognitiveclass-computer-vision-hands-on-wi-KamljRSP-py3.11/lib/python3.11/site-packages/torch/nn/__init__.py'>


In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


# Download Dataset adn Create Data Loader

In [3]:
training_data = datasets.MNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

valid_data = datasets.MNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

BATCH_SIZE: int = 64

train_dataloader: DataLoader = DataLoader(training_data, batch_size=BATCH_SIZE)
valid_dataloader: DataLoader = DataLoader(
    valid_data, batch_size=BATCH_SIZE
)

print(
    "Training data Size "
    f"{len(train_dataloader) * BATCH_SIZE}"
)
print(
    "Validation data Size "
    f"{len(valid_dataloader) * BATCH_SIZE}"
)

for X, y in valid_dataloader:
    print(
        f"Shape of X [N, C, H, W]: {X.shape}\n"
        f"Shape of y {y.shape} {y.dtype}"
    )
    break



100.0%
100.0%
100.0%
100.0%

Training data Size 60032
Validation data Size 10048
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y torch.Size([64]) torch.int64


# Define Model


We first determine the best device for perfoming training with cpu as the default device.

We the define the AI model as a Neural Network with 3 layers: an input layer, a hiddern layer,  and an output layer. Between the layers, we use a ReLU activation function.

Since the input images are 1X28x28 tensors, we need to flatten the input tensors into a 784 element tensor using the Flatten module before passing the input into our neural network.

In [4]:
device  =  torch.device(
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)

print(
    f"Using {device} device"
)



Using cpu device


In [5]:
# Define a model

class NeuralNetwork(nn.Module):
    def __init__(
        self,
        input_size,
        hidden_size,
        num_classes
    ):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(
                input_size, hidden_size
            ),
            nn.ReLU(),
            nn.Linear(
                hidden_size, hidden_size
            ),
            nn.ReLU(),
            nn.Linear(
                hidden_size, num_classes
            ),

        )
    def forward(
        self,
        image_tensor
    ):
        image_tensor = self.flatten(image_tensor)
        logits = self.linear_relu_stack(image_tensor)
        return logits


INPUT_SIZE = 28*28
HIDDEN_SIZE = 512
NUM_CLASSES = 10

model = NeuralNetwork(
    input_size=INPUT_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_classes=NUM_CLASSES
)

print('(-)'*30)
print(
    model
)
print('(-)'*30)


(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)(-)


## Training Loop

We implement a training function to use with the train_dataloader to train our model. Each iteration over the dataloader returns a batch_size image data tensor along with the expected output. After moving the tensors to the device, we call the forward pass of our model, computer the prediction error using the expected output and then call the backwards pass to compute the gradients and apply them to the model parameters.